In [6]:
# Librerias
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import re

In [2]:
# importando datos                                                        
with open('annafrank.txt') as f:
    data = f.read()
data[0:25]

'THE DIARY OF A YOUNG GIRL'

In [7]:
# Descargando recursos semanticos
nltk.download('stopwords')
sw = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/macbookpro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# Preprocesamiento
data = data.lower() # Conviertiendo texto en minuscula
data = re.sub(r'[,!?;-]', '.',data) # Eliminando caracteres raros
data = word_tokenize(data)
data = [i for i in data if i.isalpha() or i == '.'] # Eliminando caracter no alfanumericos
data = [i for i in data if i not in sw]
data = [lemmatizer.lemmatize(i) for i in data]
data[:20]

['diary',
 'young',
 'girl',
 'definitive',
 'edition',
 'anne',
 'frank',
 'edited',
 'otto',
 'frank',
 'mirjam',
 'pressler',
 'translated',
 'susan',
 'massotty',
 'book',
 'flap',
 'anne',
 'frank',
 'diary']

Posterior al preprocesamiento, se debe crear el vocabulario de las palabras unicas contenidas dentro del texto. Tal como se muestra a continuacion, el texto cuenta con 7087 palabras unicas ordenadas alfabeticamente

In [20]:
# Se crea el vocabulario de palabras
voc = set(data)
voc = sorted(voc)
print(len(voc))
print(voc[:20])

7087
['.', 'aachen', 'aagje', 'aah', 'abandon', 'abandoned', 'abduction', 'aber', 'abide', 'ability', 'abject', 'ablaze', 'able', 'abominable', 'abounds', 'aboveboard', 'abruptly', 'absent', 'absentminded', 'absolute']


hablar de :
diferentes metodos de encoding a nivel general (cuales no se van a usar)
datos a utilizar
preprocesamiento


# One Hot Encoder
El One Hot Encoder es una evolucion del indicator encoding. Este ultimo codifica cada palabra en una nueva variable hasta obtener N-1 columnas, siendo N el tamaño del vocabulario. El indicator encoding eliminaba una columna dado que las categorias son mutuamente excluyentes, por lo tanto el tener N variables podria romper el supuesto de independencia entre las palabras que se quieren modelar. Actualmente, esto no es necesario dado que que los modelos modernos generalmente incorporan metodos de regularizacion, disminuyendo la importancia de variables linealmente correlaccionadas.

A partir de lo anterior, surge el one Hot Encoder el cual es uno de los mas usados entre los metodos de conteo, el cual crea un vector para cada palabra y dentro de ese vector se asigna el valor de 1 para representar la presencia de la palabra dentro de una posicion especifica del vocabulario

$$abandono = [0, 0, 0, 0, 1, 0, 0 ..... 0, 0, 0, 0]_{300}$$

Para el ejemplo anterior se muestra la representacion vectorial de la palabra **abandono**, la cual tiene un valor de 1 en la posicion 4 (tomando como valor de inicio el 0). Es decir, dentro del vocabulario, **abandono** se encuentra en la posicion 4. Asimimo, cuando se agregan todos los vectores de las palabras del vocabulario se obtiene una matriz tal y como se muestra a continuacion:

In [30]:
# Aplicando OHE
OHE = OneHotEncoder()
OHE.fit(np.reshape(voc, (-1, 1)))
embeddingOHE = pd.DataFrame(OHE.transform(np.reshape(voc, (-1, 1))).toarray(), columns=OHE.get_feature_names_out())
embeddingOHE.head()

,x0_.,x0_aachen,x0_aagje,x0_aah,x0_abandon,x0_abandoned,x0_abduction,x0_aber,x0_abide,x0_ability,...,x0_zealand,x0_zero,x0_zeus,x0_zhlobin,x0_zionist,x0_zipper,x0_zone,x0_zookeeper,x0_zu,x0_zweite
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Tal como se muestra en la tabla, se crea una matriz simetrica de 7087x7087. El hecho de que sea simetrica indica que el vector que representa cada palabra puede obtenerse al extraer una fila o columna de la matriz. Por ejemplo, para la palabra abandono se puede extraer el vector para ambos componentes y compararlos:

In [58]:
# Extrayendo 10 primeros elementos vector palabra abandon
print("Vector por columnas:", np.array(embeddingOHE['x0_abandon'])[:10])
print("Vector por filas:", np.array(embeddingOHE.iloc[4, :])[:10])

Vector por columnas: [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
Vector por filas: [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


Debilidades en la similaridad

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])